<a href="https://colab.research.google.com/github/Adykey79/deep-learning-challenge/blob/main/Deep_Learning_Challenge/AlphabetSoupCharity_Optimization_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Supress Warnings
import warnings
warnings.simplefilter(action = "ignore")

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop( ['EIN'], axis = 1)
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df_unique = application_df.nunique().sort_values()
print(application_df_unique)

STATUS                        2
SPECIAL_CONSIDERATIONS        2
IS_SUCCESSFUL                 2
ORGANIZATION                  4
USE_CASE                      5
AFFILIATION                   6
INCOME_AMT                    9
APPLICATION_TYPE             17
CLASSIFICATION               71
ASK_AMT                    8747
NAME                      19568
dtype: int64


In [4]:
# Look at NAME value counts for binning
application_df['NAME'].value_counts()

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [5]:
# Choose a cutoff value
# create a list of NAME to be replace

NAME_replace_filter_series = application_df["NAME"].value_counts() < 10
NAME_replace_filter_series

NAME
PARENT BOOSTER USA INC                                                  False
TOPS CLUB INC                                                           False
UNITED STATES BOWLING CONGRESS INC                                      False
WASHINGTON STATE UNIVERSITY                                             False
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         False
                                                                        ...  
ST LOUIS SLAM WOMENS FOOTBALL                                            True
AIESEC ALUMNI IBEROAMERICA CORP                                          True
WEALLBLEEDRED ORG INC                                                    True
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI     True
WATERHOUSE CHARITABLE TR                                                 True
Name: count, Length: 19568, dtype: bool

In [6]:
NAME_to_replace = application_df["NAME"].value_counts()[NAME_replace_filter_series].index

In [7]:
# Replace in dataframe
for app in NAME_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(app,"Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

NAME
Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: count, Length: 223, dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [9]:
# Choose a cutoff value
# create a list of application types to be replaced
APPLICATION_TYPE_CUTOFF = 500

application_type_replace_filter_series = application_df["APPLICATION_TYPE"].value_counts() < APPLICATION_TYPE_CUTOFF
application_type_replace_filter_series


APPLICATION_TYPE
T3     False
T4     False
T6     False
T5     False
T19    False
T8     False
T7     False
T10    False
T9      True
T13     True
T12     True
T2      True
T25     True
T14     True
T29     True
T15     True
T17     True
Name: count, dtype: bool

In [10]:
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_type_replace_filter_series].index

In [11]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [12]:
application_df.loc[25:50]

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
25,Other,T4,Independent,C3000,ProductDev,Trust,1,0,N,5000,1
26,INFRAGARD NATIONAL MEMBERS ALLIANCE INC,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,1
27,POP WARNER LITTLE SCHOLARS INC,T3,CompanySponsored,C2000,Preservation,Trust,1,25000-99999,N,44004,1
28,FLORIDA STATE UNIVERSITY ALUMNI ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,0,N,5000,0
29,Other,T6,Independent,C2000,ProductDev,Trust,1,0,N,5000,1
30,Other,T6,Independent,C2000,ProductDev,Trust,1,100000-499999,N,323466,1
31,Other,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
32,Other,T7,Independent,C1000,ProductDev,Trust,1,0,N,5000,1
33,Other,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,1
34,Other,T3,Independent,C1000,Preservation,Trust,1,0,N,5000,0


In [13]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [14]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_gt_1_filter_series = application_df['CLASSIFICATION'].value_counts()>1
application_df['CLASSIFICATION'].value_counts()[classification_gt_1_filter_series]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

CLASSIFICATION_TYPE_CUTOFF = 200

classification_replace_filter_series = application_df["CLASSIFICATION"].value_counts() < CLASSIFICATION_TYPE_CUTOFF
classification_replace_filter_series
classifications_to_replace= application_df["CLASSIFICATION"].value_counts()[classification_replace_filter_series].index


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

In [17]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies["IS_SUCCESSFUL"].values
X = application_df_dummies.drop('IS_SUCCESSFUL', axis=1)

In [18]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimizer No 1 - Compile, Train and Evaluate the Model

* Number of Hidden Layers = 3

* Number of Neurons for each Layer

* hidden_nodes_layer1 = 100

* hidden_nodes_layer2 = 50

* hidden_nodes_layer2 = 25

* Activation Function used = RELU


In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, activation = "relu", input_dim = number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               26900     
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 dense_11 (Dense)            (None, 25)                1275      
                                                                 
 dense_12 (Dense)            (None, 1)                 26        
                                                                 
Total params: 33251 (129.89 KB)
Trainable params: 33251 (129.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [32]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train,epochs = 100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.4710 - accuracy: 0.7720
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4435 - accuracy: 0.7839
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4400 - accuracy: 0.7867
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4380 - accuracy: 0.7853
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4368 - accuracy: 0.7884
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4354 - accuracy: 0.7880
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4341 - accuracy: 0.7895
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4332 - accuracy: 0.7895
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4322 - accuracy: 0.7911
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4319 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4755 - accuracy: 0.7837 - 500ms/epoch - 2ms/step
Loss: 0.47550809383392334, Accuracy: 0.7836734652519226


In [24]:
# Export our model to HDF5 file
from google.colab import files
nn.save("AlphabetSoupCharity_Optimization-3.h5")
files.download("AlphabetSoupCharity_Optimization-3.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Attemp Number 3 To Optimise the Model To Acheive Accuracy Higher than 75%**

* For Optimising the Model in this attempt 'Name' column is included and used binning for rare occurrences (for less than 10 value counts).

## Result:

* Making above changes to the model increased the accuracy to 78%.

